In [1]:
import os
from dotenv import load_dotenv
load_dotenv('./.env')

True

In [2]:
import yt_dlp

def download_mp4_from_youtube(url):
    # Set the options for the download
    filename = 'infoslack.mp4'
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': filename,
        'quiet': True,
    }

    # Download the video file
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)

url = "https://www.youtube.com/watch?v=qN_2fnOPY-M"
download_mp4_from_youtube(url)

In [9]:
# refatorando para extrair somente o áudio
def download_audio(url):
    
    filename = 'infoslack-audio'
    ydl_opts = {
        'format': 'bestaudio',
        'postprocessors':[{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '128',
        }],
        'outtmpl': filename,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)
        
download_audio("https://www.youtube.com/watch?v=iw2TeYESnTk")

[youtube] Extracting URL: https://www.youtube.com/watch?v=iw2TeYESnTk
[youtube] iw2TeYESnTk: Downloading webpage
[youtube] iw2TeYESnTk: Downloading ios player API JSON
[youtube] iw2TeYESnTk: Downloading android player API JSON
[youtube] iw2TeYESnTk: Downloading m3u8 information
[info] iw2TeYESnTk: Downloading 1 format(s): 251
[download] Destination: infoslack-audio
[download] 100% of   10.89MiB in 00:00:00 at 18.48MiB/s    
[ExtractAudio] Destination: infoslack-audio.mp3
Deleting original file infoslack-audio (pass -k to keep)


In [10]:
# whisper local
import whisper

#model = whisper.load_model("base")
#result = model.transcribe("lecuninterview.mp4")
#print(result['text'])

In [11]:
from openai import OpenAI
client = OpenAI()

audio_file= open("infoslack-audio.mp3", "rb")
transcript = client.audio.transcriptions.create(
    model="whisper-1", 
    file=audio_file,
    response_format="text"
)

In [13]:
result = transcript

In [15]:
with open ('text.txt', 'w') as file:  
    file.write(result)

In [16]:
from langchain import OpenAI, LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(model_name="text-davinci-003", temperature=0)

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)

In [24]:
from langchain.docstore.document import Document

with open('text.txt') as f:
    text = f.read()

texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts]

In [27]:
docs[0]

Document(page_content='Word Embeddings são uma forma de representar palavras e textos como vetores numéricos. E a coisa mais interessante sobre esse conceito é que quando você pega palavras ou frases e converte em uma representação numérica, palavras que são numericamente semelhantes são semelhantes em significado também. E isso nos permite construir coisas como um mecanismo de busca com um nível de precisão interessante. Nesse vídeo vamos explorar um sistema de busca semântico usando Embeddings da OpenAI. Então, como funciona esse mecanismo de busca? Como pegar essas palavras e frases, converter em números para realizar classificação, detecção de anomalias, agrupamento, enfim, todas essas tarefas legais de linguagem natural? Bem, eu preparei esse exemplo sobre Embeddings de palavras e agora vamos ver como tudo isso funciona. Há matemática por trás, de uma maneira que você possa entender e aplicar. Eu vou deixar um link desse projeto na descrição. Bem, a primeira coisa que precisamos f

In [25]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

chain = load_summarize_chain(llm, chain_type="map_reduce")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

  This article explains how to use OpenAI's Word Embeddings to convert words and phrases into
numeric vectors, allowing for the creation of semantic search systems. It also discusses the
challenges of using these models in companies with large amounts of data and users with specific
intentions. The article suggests using a model of hallucination to provide satisfactory responses to
questions, with a knowledge base on specific domains, Vector Database as a pinecone, and well-
designed queries and prompts.


In [20]:
print( chain.llm_chain.prompt.template )

Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:


In [21]:
prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])

In [22]:
chain = load_summarize_chain(llm, 
                             chain_type="stuff", 
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary, 
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)


- Word Embeddings is a way to represent words and texts as numerical vectors.
- Words that are numerically similar are also similar in meaning.
- OpenAI Embeddings can be used to build a semantic search engine.
- Steps include installing OpenAI package, importing Python packages, configuring API key, and loading CSV file into a pandas dataframe.
- GetEmbedding function is used to convert words into vectors.
- Lambda function is used to apply GetEmbedding to each element in the dataframe column.
- Vector of a search term (e.g. Caputino) can be used to find similarity in numerical representations.


In [28]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

  Word Embeddings são uma forma de representar palavras e textos como vetores numéricos, permitindo
a construção de mecanismos de busca com um nível de precisão interessante. Neste vídeo, exploramos
um sistema de busca semântico usando Embeddings da OpenAI, mostrando como pegar palavras e frases,
converter em números e realizar tarefas de linguagem natural. Usando o pacote da OpenAI, importamos
os pacotes Python necessários, como o pandas e numpy. Configuramos a chave de API da OpenAI e
carregamos um arquivo CSV de palavras em um dataframe do pandas. Em seguida, calculamos os
Embeddings das palavras, o que significa converter essas palavras em vetores usando a OpenAI. Ao
enviar uma requisição para a API com uma palavra, como retorno recebemos um modelo


In [33]:
docs[0]

Document(page_content='Word Embeddings são uma forma de representar palavras e textos como vetores numéricos. E a coisa mais interessante sobre esse conceito é que quando você pega palavras ou frases e converte em uma representação numérica, palavras que são numericamente semelhantes são semelhantes em significado também. E isso nos permite construir coisas como um mecanismo de busca com um nível de precisão interessante. Nesse vídeo vamos explorar um sistema de busca semântico usando Embeddings da OpenAI. Então, como funciona esse mecanismo de busca? Como pegar essas palavras e frases, converter em números para realizar classificação, detecção de anomalias, agrupamento, enfim, todas essas tarefas legais de linguagem natural? Bem, eu preparei esse exemplo sobre Embeddings de palavras e agora vamos ver como tudo isso funciona. Há matemática por trás, de uma maneira que você possa entender e aplicar. Eu vou deixar um link desse projeto na descrição. Bem, a primeira coisa que precisamos f

In [35]:
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_ENV = os.environ.get('PINECONE_ENV')

embeddings = OpenAIEmbeddings()

In [36]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

In [37]:
index_name = "linuxtips"
index = pinecone.Index(index_name)

In [39]:
docsearch = Pinecone.from_texts([d.page_content for d in docs], embeddings, index_name=index_name, namespace='extras')

In [41]:
from langchain.llms import OpenAI
query = "o que são word embeddings?"
docs = docsearch.similarity_search(query)

In [43]:
docs[0]

Document(page_content='Word Embeddings são uma forma de representar palavras e textos como vetores numéricos. E a coisa mais interessante sobre esse conceito é que quando você pega palavras ou frases e converte em uma representação numérica, palavras que são numericamente semelhantes são semelhantes em significado também. E isso nos permite construir coisas como um mecanismo de busca com um nível de precisão interessante. Nesse vídeo vamos explorar um sistema de busca semântico usando Embeddings da OpenAI. Então, como funciona esse mecanismo de busca? Como pegar essas palavras e frases, converter em números para realizar classificação, detecção de anomalias, agrupamento, enfim, todas essas tarefas legais de linguagem natural? Bem, eu preparei esse exemplo sobre Embeddings de palavras e agora vamos ver como tudo isso funciona. Há matemática por trás, de uma maneira que você possa entender e aplicar. Eu vou deixar um link desse projeto na descrição. Bem, a primeira coisa que precisamos f

In [53]:
retriever = docsearch.as_retriever(search_type='similarity', search_kwargs={'k': 3})

In [54]:
from langchain.prompts import PromptTemplate
prompt_template = """Use as seguintes transcrições para responder à pergunta em formato de bullet points e de forma resumida. Se não souber a resposta, diga apenas que não sabe, não tente inventar uma resposta.

{context}

Question: {question}
Resposta resumida em bullter points:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [55]:
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever)

print( qa.run("Resuma word embeddings"))

 Word embeddings são uma forma de representar palavras e textos como vetores numéricos, onde palavras numericamente semelhantes são semelhantes em significado também. Isso permite construir mecanismos de busca com um nível de precisão interessante.


### Refazendo com GPT-4-turbo

In [56]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-4-1106-preview', temperature=0)

In [62]:
template="""
    Use as seguintes transcrições para responder à pergunta em formato de bullet points e de forma 
    resumida. Se não souber a resposta, diga apenas que não sabe, não tente inventar uma resposta.

    {context}

    Question: {query}
    Resposta resumida em bullter points:
"""

In [63]:
prompt = PromptTemplate(
    template=template,
    input_variables=["query", "context"]
)

In [65]:
query="Resuma word embeddings"
docs = docsearch.similarity_search(query, k=3, namespace='extras')
context = docs[0].page_content + docs[1].page_content + docs[2].page_content
res = LLMChain(prompt=prompt, llm=llm)
res.run(query=query, context=context)

'- Word Embeddings são representações vetoriais de palavras e textos.\n- Palavras com significados semelhantes têm representações numéricas próximas.\n- Essas representações permitem realizar tarefas de processamento de linguagem natural como classificação e agrupamento.\n- Embeddings podem ser obtidos através de APIs ou pacotes, como o da OpenAI.\n- Modelos como o ada002 da OpenAI podem ser usados para converter palavras em vetores.\n- Vetores de palavras podem ser armazenados em bancos de dados de vetores para consultas por similaridade.'